## 기상청 API 허브

DATA 수집 목표 : 2012 ~ (최소기간)

https://apihub.kma.go.kr/

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import urllib
import urllib.request
from urllib.parse import unquote

import json

import requests
import pprint

from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

### Text

In [ ]:
year = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
year2 = [2012, 2016, 2020]#, 2023]
# month = [1, 4, 7, 10]

In [ ]:
def short_term(year):
    url = f'https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl.php?reg=&tmfc1={year}010100&tmfc2={year+1}010100&disp=1&help=0&authKey=XZ15B2mZRGqdeQdpmURqUg'
    response = requests.get(url)
    text = response.text.split('\n')
    return text
    

In [ ]:
def short_term_half(year, a, b):
    
    url = f'https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl.php?reg=&tmfc1={year}{a}0100&tmfc2={year}{b}0100&disp=1&help=0&authKey=yEtC0d2xQmKLQtHdsWJi5g'
    response = requests.get(url)
    text = response.text.split('\n')
    return text
    

In [ ]:
url = 'https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl.php?reg=&tmfc1=2023010100&tmfc2=2023031300&disp=1&help=0&authKey=yEtC0d2xQmKLQtHdsWJi5g'
response = requests.get(url)
text = response.text.split('\n')

In [ ]:
text

In [ ]:
lst = []

for i in tqdm(year2):
    lst.append(short_term(i))

In [ ]:
# 2012, 2016, 2020 : 기간 초과

In [ ]:
year_test = [2012]

In [ ]:
lst_half = []

for i in tqdm(year2):
    lst_half.append(short_term_half(i, '01', '06'))
    lst_half.append(short_term_half(i, '07', '12'))
    

In [ ]:
for i in range(6):
    print(lst_half[i][:50])

In [ ]:
len(lst_half)

In [ ]:
lst_half[4][:-5]

In [ ]:
col_name = 'REG_ID TM_FC TM_EF        MOD NE STN C MAN_ID       MAN_FC     W1 T W2  TA  ST SKY  PREP WF'.split()
len(col_name)

In [ ]:
for i in range(8):
    print(lst[i])

tmfc1=2022060100&tmfc2=2023031300

In [ ]:
text

In [ ]:
text[-5:]

In [ ]:
lst_half[1][1]

In [ ]:
# columns 추출
col_name = lst_half[1][1].split()
col_name = col_name[1:]
print(col_name)
print(len(col_name))

In [ ]:
# col_name = [i.strip() for i in col_name]

In [ ]:
# col_name[0] = 'TM_FC'

In [ ]:
col_name

In [ ]:
# 컬럼 시작 숫자 설정
text[-2]

In [ ]:
for i in range(6):
    print(lst_half[i][-2:])

In [ ]:
text[1]

In [ ]:
df_2023 = pd.DataFrame(text[2:-2])[0].str.split(',', expand=True)
df_2023.drop(columns=17, inplace=True, axis=1)
df_2023.columns = col_name
df_2023

In [ ]:
df[9].value_counts()

In [ ]:
'''
# 띄어쓰기에 따라 나눠진 컬럼 합치기

con_cols = [18, 19, 20, 21]

df['con_cols'] = df[con_cols].apply(lambda row : ' '.join(row.values.astype(str)), axis=1)
df.drop(columns = con_cols, inplace = True)
'''

In [ ]:
df = df.iloc[:,:11]

In [ ]:
df.drop(columns = 11, inplace=True)

In [ ]:
# df_2013, df_2014, df_2015, df_2017, df_2018, df_2019, df_2021, df_2022

df1 = pd.concat([df_2013, df_2014, df_2015, df_2017, df_2018, df_2019, df_2021, df_2022])
df1.drop(columns = [17], inplace=True)
df1.columns = col_name
df1

In [ ]:
df2 = pd.concat([df_2012, df_2016, df_2020])
df2.drop(columns = [17], inplace=True)
df2.columns = col_name
df2

In [ ]:
# 필요없는 COLUMN 제거 구역코드(REG_ID), 예보관ID(MAN_ID), 예보관명(MAN_FC)
# 구간(MOD) 12, 24 시간
# SKY + PREP 한글화 = WF

df_2023.drop(columns = ['REG_ID', 'MAN_ID', 'MAN_FC'], inplace=True)

In [ ]:
df_2023

In [ ]:
df2.to_csv('단기육상예보_2차.csv')

### XML

In [ ]:
url = 'https://apihub.kma.go.kr/api/typ02/openApi/SfcYearlyInfoService/getTyphoonList?pageNo=1&numOfRows=10&dataType=XML&year=2022&authKey=_v7wW5IUQqu-8FuSFNKrLA'

response = requests.get(url)

# xml 내용
content = response.text

# 깔끔한 출력 위한 코드
pp = pprint.PrettyPrinter(indent=4)
print(pp.pprint(content))

In [ ]:
from os import name
import xml.etree.ElementTree as et
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

#bs4 사용하여 item 태그 분리
xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('info') # item 으로 구분되어 있는 경우도 있음
print(rows)

In [ ]:
# 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

In [ ]:
#xml값 DataFrame으로 만들기
xml_df = pd.DataFrame(row_list, columns=name_list)
print(xml_df.head())

In [ ]:
xml_df

In [ ]:
xml_df.to_csv('우리나라영향태풍_2021.csv')

# Data

In [2]:
df1 = pd.read_csv('H:/내 드라이브/취업/Contest/2307_PublicData/data/단기육상예보_1차.csv', index_col=0)
df2 = pd.read_csv('H:/내 드라이브/취업/Contest/2307_PublicData/data/단기육상예보_2차.csv', index_col=0)
df3 = pd.read_csv('H:/내 드라이브/취업/Contest/2307_PublicData/data/단기육상예보_3차.csv', index_col=0)

In [3]:
df1

,TM_FC,TM_EF,MOD,NE,STN,C,W1,T,W2,TA,ST,SKY,PREP,WF
0,201301010500,201301011200,A02,1,112,2,NW,1,N,1,60,DB04,3,흐리고 한때 눈
1,201301010500,201301020000,A02,2,112,2,NW,1,N,-10,70,DB04,3,흐리고 눈
2,201301010500,201301021200,A02,3,112,2,NW,1,N,-9,60,DB04,3,흐리고 한때 눈
3,201301010500,201301010000,A02,0,108,2,E,1,SE,-99,60,DB04,3,흐리고 한때 눈
4,201301010500,201301011200,A02,1,108,2,SW,1,W,-1,60,DB04,3,흐리고 가끔 눈
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253483,202209171700,202209171200,A02,0,184,2,NE,1,E,-99,60,DB04,1,흐리고 한때 비
1253484,202209171700,202209180000,A02,1,184,2,N,1,NE,24,30,DB04,0,흐림
1253485,202209171700,202209181200,A02,2,184,2,N,1,NE,27,20,DB03,0,구름많음
1253486,202209171700,202209190000,A02,3,184,2,NW,1,N,24,20,DB03,0,구름많음


In [4]:
df2

,TM_FC,TM_EF,MOD,NE,STN,C,W1,T,W2,TA,ST,SKY,PREP,WF
0,201201010500,201201011200,A02,1,112,2,NW,1,N,1,10,DB02,0,구름조금
1,201201010500,201201020000,A02,2,112,2,NW,1,N,-4,10,DB02,0,구름조금
2,201201010500,201201021200,A02,3,112,2,NW,1,N,1,20,DB03,0,구름많음
3,201201010500,201201010000,A02,0,108,2,W,1,NW,-99,20,DB03,0,구름많음
4,201201010500,201201011200,A02,1,108,2,NW,1,N,1,20,DB03,0,구름많음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491258,201205312300,201206020000,A02,3,156,2,N,1,NE,17,20,DB03,0,구름많음
491259,201205312300,201205311200,A02,0,156,2,N,1,NE,-99,10,DB02,0,구름조금
491260,201205312300,201206010000,A02,1,156,2,NW,1,N,15,10,DB02,0,구름조금
491261,201205312300,201206011200,A02,2,156,2,NW,1,N,26,20,DB03,0,구름많음


In [5]:
df3

,TM_FC,TM_EF,MOD,NE,STN,C,W1,T,W2,TA,ST,SKY,PREP,WF
0,202301010500,202301010000,A02,0,109,2,N,1,NE,-99,0,DB01,0,맑음
1,202301010500,202301011200,A02,1,109,2,NW,1,N,2,10,DB03,0,구름많음
2,202301010500,202301020000,A02,2,109,2,NW,1,N,-4,10,DB03,0,구름많음
3,202301010500,202301021200,A02,3,109,2,NW,1,N,0,10,DB01,0,맑음
4,202301010500,202301030000,A02,4,109,2,NW,1,N,-3,10,DB03,0,구름많음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342499,202303121700,202303141200,A02,4,156,2,SE,1,S,16,0,DB01,0,맑음
342500,202303121700,202303150000,A02,5,156,2,S,1,SW,6,0,DB01,0,맑음
342501,202303121700,202303151200,A02,6,156,2,NW,1,N,18,30,DB04,0,흐림
342502,202303121700,202303160000,A02,7,156,2,N,1,NE,4,30,DB04,0,흐림


In [7]:
df3['TM_FC'] = df3['TM_FC'].astype('int64')

In [9]:
df = pd.concat([df1, df2, df3]).reset_index()
df.drop(columns='index', inplace=True)
df

,TM_FC,TM_EF,MOD,NE,STN,C,W1,T,W2,TA,ST,SKY,PREP,WF
0,201301010500,201301011200,A02,1,112,2,NW,1,N,1,60,DB04,3,흐리고 한때 눈
1,201301010500,201301020000,A02,2,112,2,NW,1,N,-10,70,DB04,3,흐리고 눈
2,201301010500,201301021200,A02,3,112,2,NW,1,N,-9,60,DB04,3,흐리고 한때 눈
3,201301010500,201301010000,A02,0,108,2,E,1,SE,-99,60,DB04,3,흐리고 한때 눈
4,201301010500,201301011200,A02,1,108,2,SW,1,W,-1,60,DB04,3,흐리고 가끔 눈
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11530274,202303121700,202303141200,A02,4,156,2,SE,1,S,16,0,DB01,0,맑음
11530275,202303121700,202303150000,A02,5,156,2,S,1,SW,6,0,DB01,0,맑음
11530276,202303121700,202303151200,A02,6,156,2,NW,1,N,18,30,DB04,0,흐림
11530277,202303121700,202303160000,A02,7,156,2,N,1,NE,4,30,DB04,0,흐림


In [15]:
df = df.sort_values(by='TM_FC', ascending=True)
df = df.reset_index(drop=True)
df

,TM_FC,TM_EF,MOD,NE,STN,C,W1,T,W2,TA,ST,SKY,PREP,WF
0,201201010500,201201011200,A02,1,112,2,NW,1,N,1,10,DB02,0,구름조금
1,201201010500,201201011200,A02,1,101,2,NW,1,N,0,20,DB02,0,구름많음
2,201201010500,201201021200,A02,3,101,2,W,1,NW,0,10,DB02,0,구름조금
3,201201010500,201201011200,A02,1,112,2,NW,1,N,1,10,DB02,0,구름조금
4,201201010500,201201020000,A02,2,112,2,NW,1,N,-4,10,DB02,0,구름조금
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11530274,202303121700,202303131200,A02,2,156,2,S,1,SW,8,10,DB01,0,맑음
11530275,202303121700,202303140000,A02,3,156,2,SE,1,S,0,0,DB01,0,맑음
11530276,202303121700,202303141200,A02,4,156,2,SE,1,S,16,0,DB01,0,맑음
11530277,202303121700,202303150000,A02,5,107,2,SE,1,S,2,60,DB04,1,흐리고 한때 비


------

In [ ]:
df2 = pd.read_csv('G:/내 드라이브/취업/Contest/2307_PublicData/data/적설관측자료(2023).csv')
df2

In [ ]:
for i in df2['STN.1'].value_counts().index : print(i)

In [16]:
df.to_csv('단기육상예보.csv')

In [ ]:
df_2023.to_csv('단기육상예보_3차.csv')